In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import torch
from google.colab.patches import cv2_imshow
import zipfile
from google.colab import drive
drive.mount('/content/drive') #lo conecto con mi drive

import tensorflow as tf
from keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input

import PIL

In [ ]:
#Check to see if we have a GPU to use for training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('A {} device was detected.'.format(device))

#Print the name of the cuda device, if detected
if device == 'cuda':
  print (torch.cuda.get_device_name(device=device))

In [ ]:
directory = './data'
if not os.path.exists(directory):
    os.mkdir(directory)

# Descomprimir el zip con el dataset
with zipfile.ZipFile("./drive/MyDrive/Colab Notebooks/QuantumKernelsPlayDogsVSCats/datasetCatsVSDogs.zip","r") as z:
    z.extractall("./data")

data_dir = './data/datasetCatsVSDogs'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

In [ ]:
base_model = VGG16(weights='imagenet')
model = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)
model.summary()

In [ ]:
train_dir = "./data/datasetCatsVSDogs/train"

categorias = ['cats', 'dogs']

dataset = []

def get_features(img_path):
    img = load_img(img_path, target_size=(224, 224))
    #img = cv2.imread(imgpath,1) # el 0 indica que se cargue la imagen en escala de grises, un 1 indica en color
    #img = cv2.resize( img, (224,224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0) #la dimension de x era 3 y pasa a ser 4
    x = preprocess_input(x)
    flatten = model.predict(x) #la dimension de flatten es 2
    return list(flatten[0])

features, labels = [], []

for categoria in categorias:
  path = os.path.join(train_dir, categoria) #ruta de la carpeta cats o la carpeta dogs
  label = categorias.index(categoria)
  i=0
  for img in os.listdir(path):
    if i <1000:
      imgpath = os.path.join(path, img) #ruta de la imagen
      try:
          img = PIL.Image.open(imgpath)
      except PIL.UnidentifiedImageError:
              print(imgpath) #imprime el nombre de la imagen si se produce fallo al cargar alguna
      if (imgpath != "./data/datasetCatsVSDogs/train/cats/_DS_Store") & (imgpath != "./data/datasetCatsVSDogs/train/dogs/_DS_Store"): #este archivo esta en la carpeta pero no es una de las imagenes
        features.append(get_features(imgpath))
        labels.append(label)
    i += 1


In [ ]:


# Loop into the directory of images and extract features and labels
for image_path in dataset:
    features.append(get_features(image_path))
    labels.append(label)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split( features, labels, test_size=0.30, shuffle= True)

modelSVM = svm.SVC(C=0.5, kernel = 'poly', gamma = 'auto', probability=True) #C es la regularización


In [ ]:
modelSVM.fit(X_train, y_train) #entrena con las features y labels de los grupos train    

In [ ]:
torch.save(modelSVM, 'VGG16_SVM_definitivo.pth') #guarda el modelo en ese archivo

In [ ]:
predicted = modelSVM.predict(X_test)

In [ ]:
# get the accuracy
accuracy = accuracy_score(y_test, predicted)
print(accuracy)

In [ ]:
from PIL import Image

In [ ]:
imagePath = './data/datasetCatsVSDogs/train/dogs/dog.1897.jpg'

imagenToPredict=[]

imagenToPredict.append(get_features(imagePath)) #le debo aplicar a la imagen el mismo preprocesado que a las que se usaron para entrenar

In [ ]:
prediccion = modelSVM.predict_proba(imagenToPredict)

In [ ]:
count = 0
for i in prediccion[0]:
  if count == 0:
    catProbability = i
  else:
    dogProbability = i
  count += 1

if catProbability > dogProbability:
  classPredicted = "Cat"
else:
  classPredicted = "Dog"
print(classPredicted)

In [ ]:
print('La predicción es:', prediccion[0], ', ', categorias[prediccion[0]])

In [ ]:
#NO TOCAR NADA MÁS, ASI ESTA PERFECTO EL SCRIPT